# Classifying Observed AIA Data with Timelags and Emission Measure Slopes
Classify observed timelags with a Random Forest classifier using timelag and emission measure data

In [1]:
import os
import sys

import numpy as np
from scipy.interpolate import splev
import matplotlib
import matplotlib.pyplot as plt
from sunpy.map import Map
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ImageNormalize,SqrtStretch,AsinhStretch
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA

sys.path.append(os.path.join(os.environ['HOME'], 'projects/synthesized_timelag_maps/scripts/'))
from aiacube import DistributedAIACube
from timelags import AIATimeLags

%matplotlib inline